## 캡스톤 프로젝트 Part 1 - 항공권 특가 검색기 만들기 
- 이번 파트에서는 해당 프로그램을 만들어 우리만을 위한 저렴한 항공권 찾아보기 
- 과정은 다음과 같음 
    1. 구글 스프레드시트에 가고 싶은 장소들과 가격 상한선(저렴한 가격)을 입력 
    2. 입력한 데이터를 바탕으로 항공권 검색 API를 이용하여 향후 6개월간 가장 저렴한 항공권을 찾는다 
    3. 만약 우리가 미리 정해놓은 가겨보다 낮은 항공권이 나오면, 날짜와 가격을 Twilio SMS 모듈 혹은 SMTP 모듈을 사용하여 SMS 혹은 mail로 전달하기 

### 1. 1단계 - 구현 방향 선택 후 프로젝트 시작 

#### 각 파일 내용 확인 

In [ ]:
# data_manager.py
class DataManager:
    #This class is responsible for talking to the Google Sheet.
    pass

In [ ]:
# flight_data.py 
class FlightData:
    #This class is responsible for structuring the flight data.
    pass

In [ ]:
# flight_search.py 
class FlightSearch:
    #This class is responsible for talking to the Flight Search API.
    pass

In [ ]:
# main.py
# 여기는 프로그램 요구 사항을 충족시키기 위해 DataManager, FlightData, FlightSearch, NotificationManager 클래스를 사용해야함

In [ ]:
# notification_manager.py
class NotificationManager:
    #This class is responsible for sending notifications with the deal flight details.
    pass

#### 구글 시트 사본 만들기 
- [구글 시트 예제](https://docs.google.com/spreadsheets/d/1YMK-kYDYwuiGZoawQy7zyDjEIU9u8oggCV4H2M9j7os/edit?usp=sharing)로 들어가서 사본 만들기 

#### 필요한 API 목록
- 구글 시트 데이터 관려 - https://sheety.co/
- 키위닷컴 파트너 포탈 항공권 검색 API(무료가입 가능, 신용카드 정보 필요) - https://partners.kiwi.com/
- 키위닷컴 테킬라 항공권 검색 API 문서 - https://tequila.kiwi.com/portal/docs/tequila_api
- 트윌리오 API -  https://www.twilio.com/docs/sms
    - 트윌리오는 크레딧 제한이 있으므로 메일링으로 변경

#### 프로그램 요구사항 
- 항공권 검색 API와 Sheety API를 사용하여, 도시별 (국제항공운송협회(IATA)[https://en.wikipedia.org/wiki/IATA_airport_code#Cities_with_multiple_airports] 코드를 구글 시트 사본에 붙이기. 시트에 있는 대부분의 도시에는 공항이 여러개가 있으므로 도시 코드를 참고 
- 항공권 검색 API를 사용하여 구글 시트에 있는 모든 도시에 대해 6개월간 가장 저렴한 항공권을 확인 
- 가격이 구글 시트에 있는 최저가보다 저렴하다면, 트윌리오 API를 사용하여 본인 전화번호로 SMS 보내기 (혹은 SMTP를 사용하여 Mail 보내기)
- SMS에는 출발 공항 IATA 코드, 도착공항 IATA 코드, 출발 도시, 도착 도시, 항공권 가격, 비행 날짜가 있어야 함.

### 2. 2단계 - Sheety를 이용하여, 구글시트로 데이터 읽고 쓰기
- 해당 단계에서 데이터를 읽어와 구글 시트에 쓰는 것이 목표 
- 먼저 sheety가 제대로 동작하는지 확인한 다음, 키위 닷컴 파트너 항공권 검색 API를 연결하여 IATA 코드를 구글 시트로 가져오기 

In [14]:
# sheet API 사용 - 연결 확인
import requests
from datetime import datetime
from pprint import pprint

SHEETY_ENDPOINT = "Your End Point"

sheet_response = requests.get(url=SHEETY_ENDPOINT)
data = sheet_response.json()
data

{'prices': [{'city': 'Paris', 'iataCode': '', 'lowestPrice': 54, 'id': 2},
  {'city': 'Berlin', 'iataCode': '', 'lowestPrice': 42, 'id': 3},
  {'city': 'Tokyo', 'iataCode': '', 'lowestPrice': 485, 'id': 4},
  {'city': 'Sydney', 'iataCode': '', 'lowestPrice': 551, 'id': 5},
  {'city': 'Istanbul', 'iataCode': '', 'lowestPrice': 95, 'id': 6},
  {'city': 'Kuala Lumpur', 'iataCode': '', 'lowestPrice': 414, 'id': 7},
  {'city': 'New York', 'iataCode': '', 'lowestPrice': 240, 'id': 8},
  {'city': 'San Francisco', 'iataCode': '', 'lowestPrice': 260, 'id': 9},
  {'city': 'Cape Town', 'iataCode': '', 'lowestPrice': 378, 'id': 10},
  {'city': 'London', 'iataCode': '', 'lowestPrice': 300, 'id': 11},
  {'city': 'Taipei', 'iataCode': '', 'lowestPrice': 50, 'id': 12}]}

In [15]:
data["prices"]

[{'city': 'Paris', 'iataCode': '', 'lowestPrice': 54, 'id': 2},
 {'city': 'Berlin', 'iataCode': '', 'lowestPrice': 42, 'id': 3},
 {'city': 'Tokyo', 'iataCode': '', 'lowestPrice': 485, 'id': 4},
 {'city': 'Sydney', 'iataCode': '', 'lowestPrice': 551, 'id': 5},
 {'city': 'Istanbul', 'iataCode': '', 'lowestPrice': 95, 'id': 6},
 {'city': 'Kuala Lumpur', 'iataCode': '', 'lowestPrice': 414, 'id': 7},
 {'city': 'New York', 'iataCode': '', 'lowestPrice': 240, 'id': 8},
 {'city': 'San Francisco', 'iataCode': '', 'lowestPrice': 260, 'id': 9},
 {'city': 'Cape Town', 'iataCode': '', 'lowestPrice': 378, 'id': 10},
 {'city': 'London', 'iataCode': '', 'lowestPrice': 300, 'id': 11},
 {'city': 'Taipei', 'iataCode': '', 'lowestPrice': 50, 'id': 12}]

In [ ]:
# 파일명: data_manager.py

# 1. sheety API를 이용하여 Google Sheet에 있는 데이터 불러오기 
import requests

SHEETY_PRICES_ENDPOINT = "Your End Point"

class DataManager:
    # 클래스 내 초기화 매소드 설정 
    def __init__(self):
        self.destination_data = {}   # json 데이터를 담가 위해 초기에 설정함 
        
    def get_destination_data(self):
        # 1. Sheety API를 사용해서 google sheet에 있는 데이터 get 방식으로 가져오기 
        response = requests.get(url=SHEETY_PRICES_ENDPOINT)     # 엔드포인트 기준 get 방식으로 데이터 받기 
        data = response.json()                                  # 요청 받은 데이터를 Json 방식으로 데이터 저장
        self.destination_data = data["prices"]                  # 데이터에 prices 키 값안에 있는 데이터 받아오기 
        # 2. 로그에 출력할 JSON 데이터를 이쁘게 보여줄 pprint 사용 
        # pprint(data)
        return self.destination_data                            # destination_data 반환(get_destination_data 메소드가 호출되면 리턴해줌))

In [ ]:
# 파일명 main.py

# 2.main.py에서 데이터를 출력할 수 있도록 데이터를 다시 main.py 파일로 전달
from data_manager import DataManager

data_manager = DataManager()                      # 클래스 객체 생성 
sheet_data = data_manager.get_destination_data()  # 시트 데이터를 불러오기 위해 생성한 객체에 get_destination_data 메소드 호출
pprint(sheet_data)

In [ ]:
# flight_search.py

# 3. 항공권 정보를 얻기 위해 FlightSearch 클래스 생성 
import requests 

TEQUILA_ENDPOINT = "https://tequila-api.kiwi.com"
TEQUILA_API_KEY = "YOUR_API_KEY_HERE"

class FlightSearch:
    
    def get_destination_code(self, city_name):
        # Test용으로 "TESTING"을 반환하여 sheety가 동작하는지 확인!
        code = "TESTING"
        return code 

In [ ]:
# 파일명 main.py

# 4.main.py에서 데이터를 출력할 수 있도록 데이터를 다시 main.py 파일로 전달
from data_manager import DataManager
from flight_search import FlightSearch

data_manager = DataManager()                      # 클래스 객체 생성 
sheet_data = data_manager.get_destination_data()  # 시트 데이터를 불러오기 위해 생성한 객체에 get_destination_data 메소드 호출
# pprint(sheet_data)

if sheet_data[0]["iataCode"] == "":        # sheet_data에 "iataCode"의 키값이 없는 경우 
    flight_search = FlightSearch()         # FlightSearch 클래스 객체 선언 (Flight Search API를 사용을 위한)
    for row in sheet_data:                 
        row["iataCode"] = flight_search.get_destination_code(row['city'])  # sheet_data의 각 도시 이름을 하나씩 전달 
    print(f"sheet_data:\n {sheet_data}")

In [ ]:
# 파일명: data_manager.py

import requests

SHEETY_PRICES_ENDPOINT = "Your End Point"

class DataManager:
    # 클래스 내 초기화 매소드 설정 
    def __init__(self):
        self.destination_data = {}   # json 데이터를 담가 위해 초기에 설정함 
    
    # google sheet에 데이터를 불러오기 위한 메소드
    def get_destination_data(self):
        # 1. Sheety API를 사용해서 google sheet에 있는 데이터 get 방식으로 가져오기 
        response = requests.get(url=SHEETY_PRICES_ENDPOINT)     # 엔드포인트 기준 get 방식으로 데이터 받기 
        data = response.json()                                  # 요청 받은 데이터를 Json 방식으로 데이터 저장
        self.destination_data = data["prices"]                  # 데이터에 prices 키 값안에 있는 데이터 받아오기 
        # 2. 로그에 출력할 JSON 데이터를 이쁘게 보여줄 pprint 사용 
        # pprint(data)
        return self.destination_data                            # destination_data 반환(get_destination_data 메소드가 호출되면 리턴해줌))
    
    # 5. PUT요청을 위한 update_destination_codes 메소드 만들기
    # PUT 방식으로 데이터를 업데이트 하기 위한 메소드 
    def update_destination_codes(self):
        for city in self.destination_data:                      
            new_data = {                                        # get 방식으로 받은 데이터에서 city IATA CODE를 저장 
                "price":{
                    "iataCode": city["iataCode"]
                }
            }
            
            response = requests.put(                           # 데이터를 수정하기 위해 PUT 방식으로 호출 
            url=f"{SHEETY_PRICES_ENDPOINT}/{city['id']}",      # end point url 설정 
            json=new_data                                      # 수정할 데이터 지정 
            )
            print(response.text)                               # 결과 출력 

In [ ]:
# 파일명 main.py

# 6. IATA CODE를 고치기 위해 update_destination_codes 메소드 사용
from data_manager import DataManager
from flight_search import FlightSearch

data_manager = DataManager()                      # 클래스 객체 생성 
sheet_data = data_manager.get_destination_data()  # 시트 데이터를 불러오기 위해 생성한 객체에 get_destination_data 메소드 호출
# pprint(sheet_data)

if sheet_data[0]["iataCode"] == "":        # sheet_data에 "iataCode"의 키값이 없는 경우 
    flight_search = FlightSearch()         # FlightSearch 클래스 객체 선언 (Flight Search API를 사용을 위한)
    for row in sheet_data:                 
        row["iataCode"] = flight_search.get_destination_code(row['city'])  # sheet_data의 각 도시 이름을 하나씩 전달 
    print(f"sheet_data:\n {sheet_data}")
    
    data_manager.destination_data = sheet_data       # 수정할 Sheet 데이터 불러오기 
    data_manager.update_destination_codes()          # IATA 코드 수정 하기